In [10]:
# İmport necessary libraries
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings("ignore")

In [11]:
#Read and ddrop Unnamed:0 column.
df = pd.read_csv("eom_all (2).csv")
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [12]:
#Convert trd_exctn_dt column to datetime format.
df["trd_exctn_dt"] = pd.to_datetime(df["trd_exctn_dt"])
#Clean rows if INTEREST_FREQUENCY is not bigger than 0.
df = df[df["INTEREST_FREQUENCY"] >= 0]

In [13]:
zeros = df[df["INTEREST_FREQUENCY"] == 0]

In [14]:
#Drop rows that contains NaN
df.dropna(inplace=True)
#Drop cusip id's that has only 1 row.
df = df[df.groupby("cusip_id").cusip_id.transform("count") > 1]

In [15]:
#Sort values and reset-set index.
df = df.sort_values("cusip_id")
df = df.reset_index().sort_values("index")
df.set_index("index", inplace=True) 

In [16]:
#Convert INTEREST_FREQUENCY column to appropriate format.
df["INTEREST_FREQUENCY"] = df["INTEREST_FREQUENCY"].astype("int")
df["INTEREST_FREQUENCY"] = df["INTEREST_FREQUENCY"].astype("str")
df["INTEREST_FREQUENCY"].replace({"2": "6M", "4": "3M", "12": "1M"}, inplace=True)

In [17]:
# Function that clears non-consecutive rows for each cusip id.
def Cleaner(df):
 qf = pd.DataFrame()
 for mf in df['cusip_id'].unique():
    
    zf = df[df['cusip_id'] == mf]
    
    dt = zf['trd_exctn_dt']
    
    mzf = zf.loc[(((dt.dt.month - dt.shift(-1).dt.month).abs() == 1)  |  (dt.dt.month - dt.shift(1).dt.month.abs() == 1))
                | ((dt.dt.month - dt.shift(-1).dt.month).abs() == 11)  | ((dt.dt.year - dt.shift(1).dt.year) == 1 )]   
   
    qf=pd.concat([qf,mzf],axis=0)
 return qf

# Creating artificial dataframe for checking functions results.

In [18]:
check = df.iloc[0:13,:]
paid = pd.date_range(start='2021-08-08', end='2023-10-08', inclusive="both", freq='1M')

In [19]:
#3 different consecutive strings that contains the year change.
paid1 = paid[2:6]
paid4 = paid[7:8]
paid2 = paid[10:13]
paid3= paid[15:20]

In [20]:
dates = paid1.append([paid2,paid4,paid3])

In [21]:
dates

DatetimeIndex(['2021-10-31', '2021-11-30', '2021-12-31', '2022-01-31',
               '2022-06-30', '2022-07-31', '2022-08-31', '2022-03-31',
               '2022-11-30', '2022-12-31', '2023-01-31', '2023-02-28',
               '2023-03-31'],
              dtype='datetime64[ns]', freq=None)

In [22]:
check['trd_exctn_dt'] = dates   
check['cusip_id'] = '00138GAA7'

# Uncleaned state.

In [23]:
check

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price
index,,,,,,,,,,,
0,00138GAA7,2021-10-31,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,146.937000,1000.0,147.397417
1,00138GAA7,2021-11-30,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,139.235000,1000.0,140.285694
2,00138GAA7,2021-12-31,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,138.031500,1000.0,139.353722
3,00138GAA7,2022-01-31,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,137.126000,1000.0,138.164889
4,00138GAA7,2022-06-30,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,140.721333,1000.0,142.173417
5,00138GAA7,2022-07-31,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,138.706111,1000.0,140.559583
6,00138GAA7,2022-08-31,19940215.0,19940815.0,6M,20230815.0,20240215.0,7.375,120.333500,1000.0,120.548604
7,00138GAA7,2022-03-31,19940215.0,19940815.0,6M,20230815.0,20240215.0,7.375,119.367333,1000.0,119.705354
8,00138GAA7,2022-11-30,19940215.0,19940815.0,6M,20230815.0,20240215.0,7.375,117.227889,1000.0,117.688827


In [24]:
check = Cleaner(check)

# Cleaned state.

In [25]:
check

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price
index,,,,,,,,,,,
0,00138GAA7,2021-10-31,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,146.937000,1000.0,147.397417
1,00138GAA7,2021-11-30,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,139.235000,1000.0,140.285694
2,00138GAA7,2021-12-31,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,138.031500,1000.0,139.353722
3,00138GAA7,2022-01-31,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,137.126000,1000.0,138.164889
4,00138GAA7,2022-06-30,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,140.721333,1000.0,142.173417
5,00138GAA7,2022-07-31,20130711.0,20140101.0,6M,20300101.0,20300701.0,8.500,138.706111,1000.0,140.559583
6,00138GAA7,2022-08-31,19940215.0,19940815.0,6M,20230815.0,20240215.0,7.375,120.333500,1000.0,120.548604
8,00138GAA7,2022-11-30,19940215.0,19940815.0,6M,20230815.0,20240215.0,7.375,117.227889,1000.0,117.688827
9,00138GAA7,2022-12-31,19940215.0,19940815.0,6M,20230815.0,20240215.0,7.375,115.732000,1000.0,117.053354


In [26]:
# Convert FIRST_INTEREST_DATE and LAST_INTEREST_DATE to datetime format.
df["FIRST_INTEREST_DATE"] = df["FIRST_INTEREST_DATE"].astype(int)
df["LAST_INTEREST_DATE"] = df["LAST_INTEREST_DATE"].astype(int)

df["FIRST_INTEREST_DATE"] = pd.to_datetime(df.FIRST_INTEREST_DATE, format="%Y%m%d")
df["LAST_INTEREST_DATE"] = pd.to_datetime(df.LAST_INTEREST_DATE, format="%Y%m%d")

In [27]:
# Reset index again because after data cleaning indexes changed.
df = df.reset_index()
df.drop(["index"], axis=1, inplace=True)

# Setting artifical DataFrame for testing C columns function on it.

In [28]:
check['INTEREST_FREQUENCY'] = '6M'

In [29]:
check['C'] = 0 

In [30]:
check["FIRST_INTEREST_DATE"] = check["FIRST_INTEREST_DATE"].astype(int)
check["LAST_INTEREST_DATE"] = check["LAST_INTEREST_DATE"].astype(int)
check["FIRST_INTEREST_DATE"] = pd.to_datetime(check.FIRST_INTEREST_DATE, format="%Y%m%d")
check["LAST_INTEREST_DATE"] = pd.to_datetime(check.LAST_INTEREST_DATE, format="%Y%m%d")
check = check.reset_index()
check.drop(["index"], axis=1, inplace=True)

# Just For Testing

In [31]:
#define dataframe for work on it.
zf = pd.DataFrame()
for m in check["cusip_id"].unique():
    
    #df for unique cusip_id 
    mf = check.loc[check["cusip_id"] == m]    
    
    #pd.Series for dates
    dates = mf['trd_exctn_dt']    
    
    #Defining dates.
    start = mf.FIRST_INTEREST_DATE.iloc[0]
    end = mf.LAST_INTEREST_DATE.iloc[0]
    freq = mf.INTEREST_FREQUENCY.iloc[0]
   
    paid = pd.date_range(start=start, end=end, inclusive="both", freq=freq)
    
    #for each pair of dates in the dates series
    
    for i in range(dates.shape[0]-1):
        pastmonth, pastyear = dates[i].month, dates[i].year
        nowmonth, nowyear = dates[i+1].month, dates[i+1].year
        
        # if date pair is suitable to work on 
        if ((pastmonth + 1 == nowmonth) & (pastyear == nowyear)) |  ((pastyear +1 == nowyear) & (pastmonth-11 == nowmonth)):
            #Check if the paid dates fall within the appropriate ranges.
            for j in paid:
                #For those who fall within certain ranges from the paid dates, make column c of the larger date in the pair 1
                if (dates[i] <= j <= dates[i+1]) == 1 :                     
                    mf["C"].loc[mf["trd_exctn_dt"] == dates [i+1] ] = 1
    
    zf = pd.concat([zf, mf], axis=0)  

In [32]:
zf

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price,C
0,00138GAA7,2021-10-31,20130711.0,2014-01-01,6M,2030-01-01,20300701.0,8.500,146.937000,1000.0,147.397417,0
1,00138GAA7,2021-11-30,20130711.0,2014-01-01,6M,2030-01-01,20300701.0,8.500,139.235000,1000.0,140.285694,0
2,00138GAA7,2021-12-31,20130711.0,2014-01-01,6M,2030-01-01,20300701.0,8.500,138.031500,1000.0,139.353722,0
3,00138GAA7,2022-01-31,20130711.0,2014-01-01,6M,2030-01-01,20300701.0,8.500,137.126000,1000.0,138.164889,1
4,00138GAA7,2022-06-30,20130711.0,2014-01-01,6M,2030-01-01,20300701.0,8.500,140.721333,1000.0,142.173417,0
5,00138GAA7,2022-07-31,20130711.0,2014-01-01,6M,2030-01-01,20300701.0,8.500,138.706111,1000.0,140.559583,1
6,00138GAA7,2022-08-31,19940215.0,1994-08-15,6M,2023-08-15,20240215.0,7.375,120.333500,1000.0,120.548604,1
7,00138GAA7,2022-11-30,19940215.0,1994-08-15,6M,2023-08-15,20240215.0,7.375,117.227889,1000.0,117.688827,0
8,00138GAA7,2022-12-31,19940215.0,1994-08-15,6M,2023-08-15,20240215.0,7.375,115.732000,1000.0,117.053354,0
9,00138GAA7,2023-01-31,19940215.0,1994-08-15,6M,2023-08-15,20240215.0,7.375,114.964000,1000.0,115.394208,1


In [33]:
df = Cleaner(df)

In [34]:
df['C'] = 0

In [35]:
#define dataframe for work on it.
zf = pd.DataFrame()
count=0
for m in df["cusip_id"].unique():

    
    #pd.Series for dates
    dates = mf['trd_exctn_dt'].reset_index(drop=True) 
    
    #df for unique cusip_id 
    mf = df.loc[df["cusip_id"] == m]    
    
    mf = mf.sort_values('trd_exctn_dt')
    
    #Defining dates.
    start = mf.FIRST_INTEREST_DATE.iloc[0]
    end = mf.LAST_INTEREST_DATE.iloc[0]
    freq = mf.INTEREST_FREQUENCY.iloc[0]
   
    paid = pd.date_range(start=start, end=end, inclusive="both", freq=freq)
  
    #for each pair of dates in the dates series

    for i in range(dates.shape[0]-1):
        pastmonth, pastyear = dates[i].month, dates[i].year
        nowmonth, nowyear = dates[i+1].month, dates[i+1].year
        
        # if date pair is suitable to work on 
        if ((pastmonth + 1 == nowmonth) & (pastyear == nowyear)) |  ((pastyear +1 == nowyear) & (pastmonth-11 == nowmonth)):
            #Check if the paid dates fall within the appropriate ranges.
            for j in paid:
                #For those who fall within certain ranges from the paid dates, make column c of the larger date in the pair 1
                if (dates[i] < j < dates[i+1]) == 1 :  
                    mf["C"].loc[mf["trd_exctn_dt"] == dates[i+1]] = 1
                    
    
    zf = pd.concat([zf, mf], axis=0)  

In [36]:
zf.C.value_counts()

0    6131
1     263
Name: C, dtype: int64

In [37]:
#replace INTEREST_FREQUENCY columns values again for being able to perform calculations on it.
zf["INTEREST_FREQUENCY"].replace({"6M": "2", "3M": "4", "1M": "12"}, inplace=True)

In [38]:
#Convert object type to integer.
zf['INTEREST_FREQUENCY']  = zf['INTEREST_FREQUENCY'].astype('int')

In [39]:
#Calculate coupon_amt column.
zf['coupon_amt'] = (zf['COUPON'] * zf['PRINCIPAL_AMT']) / (zf['INTEREST_FREQUENCY'] * 1000)

In [40]:
#Calculate total_value.
zf['total_value'] = zf['dirty_price'] + zf['coupon_amt']

In [41]:
zf

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price,C,coupon_amt,total_value
0,00138GAA7,2021-02-08,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,146.937000,1000.0,147.397417,0,4.2500,151.647417
1,00138GAA7,2021-03-29,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,139.235000,1000.0,140.285694,0,4.2500,144.535694
2,00138GAA7,2021-04-21,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,138.031500,1000.0,139.353722,0,4.2500,143.603722
4,00138GAA7,2021-11-02,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,140.721333,1000.0,142.173417,0,4.2500,146.423417
5,00138GAA7,2021-12-06,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,138.706111,1000.0,140.559583,0,4.2500,144.809583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6713,989822AA9,2021-08-25,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,112.963000,1000.0,114.269250,0,3.5625,117.831750
6714,989822AA9,2021-09-27,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,112.439667,1000.0,114.062583,0,3.5625,117.625083
6715,989822AA9,2021-10-27,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,111.964333,1000.0,112.389854,0,3.5625,115.952354
6716,989822AA9,2021-11-22,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,111.102636,1000.0,111.488574,0,3.5625,115.051074


In [42]:
#Set a value for returns.
zf['returns'] =-99

In [43]:
#define dataframe for work on it.
mzf = pd.DataFrame()

for m in zf["cusip_id"].unique():

    #df for unique cusip_id 
    mf = zf.loc[zf["cusip_id"] == m]    
    
    #pd.Series for dates
    dates = mf['trd_exctn_dt'].reset_index(drop=True) 
    
    for i in range(dates.shape[0]-1):
        pastmonth, pastyear = dates[i].month, dates[i].year
        nowmonth, nowyear = dates[i+1].month, dates[i+1].year
        
        # if date pair is suitable to work on 
        if ((pastmonth + 1 == nowmonth) & (pastyear == nowyear)) |  ((pastyear +1 == nowyear) & (pastmonth-11 == nowmonth)):
            past = mf.loc[mf["trd_exctn_dt"] == dates[i]]['total_value'].iloc[0]
            now = mf.loc[mf["trd_exctn_dt"] == dates[i+1]]['total_value'].iloc[0]           
            mf["returns"].loc[mf["trd_exctn_dt"] == dates[i+1]] = now/past
            
    mzf = pd.concat([mzf, mf], axis=0)  

In [44]:
mzf

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price,C,coupon_amt,total_value,returns
0,00138GAA7,2021-02-08,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,146.937000,1000.0,147.397417,0,4.2500,151.647417,-99.000000
1,00138GAA7,2021-03-29,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,139.235000,1000.0,140.285694,0,4.2500,144.535694,0.953104
2,00138GAA7,2021-04-21,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,138.031500,1000.0,139.353722,0,4.2500,143.603722,0.993552
4,00138GAA7,2021-11-02,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,140.721333,1000.0,142.173417,0,4.2500,146.423417,-99.000000
5,00138GAA7,2021-12-06,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,138.706111,1000.0,140.559583,0,4.2500,144.809583,0.988978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6713,989822AA9,2021-08-25,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,112.963000,1000.0,114.269250,0,3.5625,117.831750,0.999119
6714,989822AA9,2021-09-27,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,112.439667,1000.0,114.062583,0,3.5625,117.625083,0.998246
6715,989822AA9,2021-10-27,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,111.964333,1000.0,112.389854,0,3.5625,115.952354,0.985779
6716,989822AA9,2021-11-22,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,111.102636,1000.0,111.488574,0,3.5625,115.051074,0.992227


In [45]:
df_filtered = mzf[mzf['returns'] != -99]

In [46]:
df_filtered

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price,C,coupon_amt,total_value,returns
1,00138GAA7,2021-03-29,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,139.235000,1000.0,140.285694,0,4.2500,144.535694,0.953104
2,00138GAA7,2021-04-21,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,138.031500,1000.0,139.353722,0,4.2500,143.603722,0.993552
5,00138GAA7,2021-12-06,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,138.706111,1000.0,140.559583,0,4.2500,144.809583,0.988978
7,001814AQ5,2021-02-16,19940215.0,1994-08-15,2,2023-08-15,20240215.0,7.375,119.367333,1000.0,119.705354,0,3.6875,123.392854,0.993213
8,001814AQ5,2021-03-29,19940215.0,1994-08-15,2,2023-08-15,20240215.0,7.375,117.227889,1000.0,117.688827,1,3.6875,121.376327,0.983658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6713,989822AA9,2021-08-25,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,112.963000,1000.0,114.269250,0,3.5625,117.831750,0.999119
6714,989822AA9,2021-09-27,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,112.439667,1000.0,114.062583,0,3.5625,117.625083,0.998246
6715,989822AA9,2021-10-27,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,111.964333,1000.0,112.389854,0,3.5625,115.952354,0.985779
6716,989822AA9,2021-11-22,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,111.102636,1000.0,111.488574,0,3.5625,115.051074,0.992227


# Zeros Part

In [47]:
zeros['C'] = 0

In [48]:
zeros['coupon_amt'] = 0 

In [49]:
zeros['total_value'] = zeros['daily_clean_pr']

In [50]:
zeros['returns'] = -99

In [51]:
#define dataframe for work on it.
mzf = pd.DataFrame()

for m in zeros["cusip_id"].unique():

    #df for unique cusip_id 
    mf = zeros.loc[zeros["cusip_id"] == m]    
    
    #pd.Series for dates
    dates = mf['trd_exctn_dt'].reset_index(drop=True) 
    
    for i in range(dates.shape[0]-1):
        pastmonth, pastyear = dates[i].month, dates[i].year
        nowmonth, nowyear = dates[i+1].month, dates[i+1].year
        
        # if date pair is suitable to work on 
        if ((pastmonth + 1 == nowmonth) & (pastyear == nowyear)) |  ((pastyear +1 == nowyear) & (pastmonth-11 == nowmonth)):
            past = mf.loc[mf["trd_exctn_dt"] == dates[i]]['total_value'].iloc[0]
            now = mf.loc[mf["trd_exctn_dt"] == dates[i+1]]['total_value'].iloc[0]           
            mf["returns"].loc[mf["trd_exctn_dt"] == dates[i+1]] = now/past
            
    mzf = pd.concat([mzf, mf], axis=0)  

In [52]:
df_filtered_zeros = mzf[mzf['returns'] != -99]

In [53]:
df_filtered_zeros

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price,C,coupon_amt,total_value,returns
3884,48130UPE9,2021-02-10,20181019.0,NaN,0.0,NaN,20231016.0,0.0,212.480600,1000.0,NaN,0,0,212.480600,0.953384
3885,48130UPE9,2021-03-24,20181019.0,NaN,0.0,NaN,20231016.0,0.0,188.570000,1000.0,NaN,0,0,188.570000,0.887469
3886,48130UPE9,2021-04-28,20181019.0,NaN,0.0,NaN,20231016.0,0.0,208.840000,1000.0,NaN,0,0,208.840000,1.107493
3887,48130UPE9,2021-05-26,20181019.0,NaN,0.0,NaN,20231016.0,0.0,197.500000,1000.0,NaN,0,0,197.500000,0.945700
3888,48130UPE9,2021-06-23,20181019.0,NaN,0.0,NaN,20231016.0,0.0,208.680000,1000.0,NaN,0,0,208.680000,1.056608
3889,48130UPE9,2021-07-28,20181019.0,NaN,0.0,NaN,20231016.0,0.0,226.090000,1000.0,NaN,0,0,226.090000,1.083429
3890,48130UPE9,2021-08-16,20181019.0,NaN,0.0,NaN,20231016.0,0.0,233.590000,1000.0,NaN,0,0,233.590000,1.033173
3891,48130UPE9,2021-09-28,20181019.0,NaN,0.0,NaN,20231016.0,0.0,221.990000,1000.0,NaN,0,0,221.990000,0.950340
3892,48130UPE9,2021-10-27,20181019.0,NaN,0.0,NaN,20231016.0,0.0,229.933333,1000.0,NaN,0,0,229.933333,1.035782
3893,48130UPE9,2021-11-03,20181019.0,NaN,0.0,NaN,20231016.0,0.0,233.388333,1000.0,NaN,0,0,233.388333,1.015026


In [54]:
df_filtered

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price,C,coupon_amt,total_value,returns
1,00138GAA7,2021-03-29,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,139.235000,1000.0,140.285694,0,4.2500,144.535694,0.953104
2,00138GAA7,2021-04-21,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,138.031500,1000.0,139.353722,0,4.2500,143.603722,0.993552
5,00138GAA7,2021-12-06,20130711.0,2014-01-01,2,2030-01-01,20300701.0,8.500,138.706111,1000.0,140.559583,0,4.2500,144.809583,0.988978
7,001814AQ5,2021-02-16,19940215.0,1994-08-15,2,2023-08-15,20240215.0,7.375,119.367333,1000.0,119.705354,0,3.6875,123.392854,0.993213
8,001814AQ5,2021-03-29,19940215.0,1994-08-15,2,2023-08-15,20240215.0,7.375,117.227889,1000.0,117.688827,1,3.6875,121.376327,0.983658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6713,989822AA9,2021-08-25,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,112.963000,1000.0,114.269250,0,3.5625,117.831750,0.999119
6714,989822AA9,2021-09-27,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,112.439667,1000.0,114.062583,0,3.5625,117.625083,0.998246
6715,989822AA9,2021-10-27,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,111.964333,1000.0,112.389854,0,3.5625,115.952354,0.985779
6716,989822AA9,2021-11-22,19931020.0,1994-04-15,2,2023-04-15,20231015.0,7.125,111.102636,1000.0,111.488574,0,3.5625,115.051074,0.992227


# Concat 

In [55]:
df_final = pd.concat([df_filtered,df_filtered_zeros],axis=0)

In [56]:
df_final

,cusip_id,trd_exctn_dt,DATED_DATE,FIRST_INTEREST_DATE,INTEREST_FREQUENCY,LAST_INTEREST_DATE,MATURITY,COUPON,daily_clean_pr,PRINCIPAL_AMT,dirty_price,C,coupon_amt,total_value,returns
1,00138GAA7,2021-03-29,20130711.0,2014-01-01 00:00:00,2.0,2030-01-01 00:00:00,20300701.0,8.500,139.235000,1000.0,140.285694,0,4.2500,144.535694,0.953104
2,00138GAA7,2021-04-21,20130711.0,2014-01-01 00:00:00,2.0,2030-01-01 00:00:00,20300701.0,8.500,138.031500,1000.0,139.353722,0,4.2500,143.603722,0.993552
5,00138GAA7,2021-12-06,20130711.0,2014-01-01 00:00:00,2.0,2030-01-01 00:00:00,20300701.0,8.500,138.706111,1000.0,140.559583,0,4.2500,144.809583,0.988978
7,001814AQ5,2021-02-16,19940215.0,1994-08-15 00:00:00,2.0,2023-08-15 00:00:00,20240215.0,7.375,119.367333,1000.0,119.705354,0,3.6875,123.392854,0.993213
8,001814AQ5,2021-03-29,19940215.0,1994-08-15 00:00:00,2.0,2023-08-15 00:00:00,20240215.0,7.375,117.227889,1000.0,117.688827,1,3.6875,121.376327,0.983658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3890,48130UPE9,2021-08-16,20181019.0,NaN,0.0,NaN,20231016.0,0.000,233.590000,1000.0,NaN,0,0.0000,233.590000,1.033173
3891,48130UPE9,2021-09-28,20181019.0,NaN,0.0,NaN,20231016.0,0.000,221.990000,1000.0,NaN,0,0.0000,221.990000,0.950340
3892,48130UPE9,2021-10-27,20181019.0,NaN,0.0,NaN,20231016.0,0.000,229.933333,1000.0,NaN,0,0.0000,229.933333,1.035782
3893,48130UPE9,2021-11-03,20181019.0,NaN,0.0,NaN,20231016.0,0.000,233.388333,1000.0,NaN,0,0.0000,233.388333,1.015026
